In [1]:
!pip list

Package                  Version
------------------------ -----------
aiohappyeyeballs         2.6.1
aiohttp                  3.12.13
aiosignal                1.4.0
annotated-types          0.7.0
anyio                    4.9.0
asttokens                3.0.0
attrs                    25.3.0
certifi                  2025.6.15
charset-normalizer       3.4.2
colorama                 0.4.6
comm                     0.2.2
dataclasses-json         0.6.7
debugpy                  1.8.11
decorator                5.2.1
distro                   1.9.0
exceptiongroup           1.3.0
executing                2.2.0
faiss-cpu                1.11.0
filelock                 3.18.0
frozenlist               1.7.0
fsspec                   2025.5.1
graphviz                 0.21
greenlet                 3.2.3
h11                      0.16.0
httpcore                 1.0.9
httpx                    0.28.1
httpx-sse                0.4.1
huggingface-hub          0.33.2
idna                     3.10
importlib_metadat

In [158]:
from typing import TypedDict, List, Annotated
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import os
from langgraph.graph import StateGraph
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.retrievers import BM25Retriever, EnsembleRetriever
import cohere

In [ ]:
# 환경 변수 설정
load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")  

# FAISS 벡터 DB 불러오기
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")
vectorstore = FAISS.load_local("faiss_card_db_alldocuments", embedding_model,allow_dangerous_deserialization=True)


NameError: name 'BM25Retriever' is not defined

In [ ]:
import getpass
# os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")
COHERE_API_KEY = os.environ["COHERE_API_KEY"]
print("Cohere API Key:", COHERE_API_KEY)  
co = cohere.Client(COHERE_API_KEY)


Cohere API Key: XKTCFN7i37z9ZTOSUmwfjBjOe65wekLzjbawsZqg


In [316]:
# 전체 문서 (BM25용)
all_docs = vectorstore.docstore._dict.values()

# BM25 retriever
bm25_retriever = BM25Retriever.from_documents(all_docs)
bm25_retriever.k = 5

# faiss retriever
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Hybrid (ensemble) retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.5, 0.5]
)

In [ ]:
# 1. State 정의
class GraphState(TypedDict):
    question: Annotated[str, "질문"]
    answer: Annotated[str, "답변"]
    score: Annotated[float, "유사도 점수"]
    retriever_docs: Annotated[List[Document], "유사도 상위문서"]

# 2. 노드 정의
def retriever_node(state: GraphState) -> GraphState:
    docs = vectorstore.similarity_search_with_score(state["question"], k=3)
    retrieved_docs = [doc for doc, _ in docs]
    score = docs[0][1]
    # print("\n[retriever_node] 문서:", [doc.page_content for doc in retrieved_docs])
    # print("[queston]:", state["question"])
    # print("[retriever_node] faiss 리트리버 상위 문서의 제목:\n",docs)
    # print("[retriever_node] faiss 거리 기반 유사도 점수:", score)
    # print('-'*100)
    # print(docs)
    # print('-'*100)
    # print(retrieved_docs[0])

    hybrid_docs = ensemble_retriever.invoke(state["question"])
  
    print("앙상블리트리버에서 invoke 로 가져온 결과\n",hybrid_docs)
    hybrid_docs = [doc for doc in hybrid_docs if doc.page_content]  # 빈 문서 제거
    # hybrid_docs = sorted(hybrid_docs, key=lambda x: x.metadata.get("score", 0), reverse=True)[:10]  
    
    # Cohere Rerank용 문서 구성
    rerank_docs = [
        {
            "text": doc.page_content.strip(),
            "metadata": doc.metadata,
            "id": doc.metadata.get("source", f"doc_{i}")
        }
        for i, doc in enumerate(hybrid_docs)
        if doc.page_content.strip()
    ]


    rerank_result = co.rerank(
        model="rerank-multilingual-v3.0",
        query=state["question"],
        documents=rerank_docs,
        top_n=3,
        return_documents=True
    )

    reranked_docs = [
        Document(
            page_content=res.document.text,
            metadata={**res.document.metadata, "score": res.relevance_score}
        )
        for res in rerank_result.results
    ]

    # print("[retriever_node] rerank_result 결과:", rerank_result)
    print("-"*10000)
    print("[retriever_node] reranked_docs", reranked_docs)

    return GraphState(score=score, retriever_docs=reranked_docs)

def grade_documents_node(state: GraphState) -> GraphState:
    return GraphState()

def llm_answer_node(state: GraphState) -> GraphState:
    prompt = ChatPromptTemplate.from_template(
        """
        문서: {docs}
        질문: {question}
        위 문서들을 참고해서 질문에 답변해줘.
        """
    )
    docs_content = "\n---\n".join([doc.page_content for doc in state["retriever_docs"]])
    chain = prompt | ChatOpenAI(model="gpt-4.1-mini-2025-04-14")
    answer = chain.invoke({"docs": docs_content, "question": state["question"]}).content
    print("\n[llm_answer_node] 생성된 답변:", answer)
    return GraphState(answer=answer)

def query_rewrite_node(state: GraphState) -> GraphState:
    prompt = ChatPromptTemplate.from_template(
        """
        원본 질문: {question}
        위 질문의 핵심은 유지하면서, 유사 문서를 더 잘 찾을 수 있도록 질문을 다시 써줘.
        """
    )
    chain = prompt | ChatOpenAI(model="gpt-4.1-mini-2025-04-14")
    new_question = chain.invoke({"question": state["question"]}).content
    print("\n[query_rewrite_node] :", new_question)
    return GraphState(question=new_question)

# 3. 노드 분기 함수 정의
def decide_to_generate(state: GraphState) -> str:
    """
    문서의 유사도 점수에 따라 다음 노드를 결정
    - score가 0.23 이하면 'llm_answer'로 이동
    - score가 0.23 초과면 'query_rewrite'로 이동
    """
    if state["score"] <= 0.5:
        return "llm_answer"
    else:
        return "query_rewrite"


# 4. LangGraph 구성 및 연결
workflow = StateGraph(GraphState)
workflow.add_node("retriever", retriever_node)
workflow.add_node("grade_documents", grade_documents_node)
workflow.add_node("llm_answer", llm_answer_node)
workflow.add_node("query_rewrite", query_rewrite_node)
workflow.set_entry_point("retriever")
workflow.add_edge("retriever", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "llm_answer": "llm_answer",
        "query_rewrite": "query_rewrite",
    },
)
workflow.add_edge("query_rewrite", "retriever")

In [321]:
# 실행 
app = workflow.compile()

response = app.invoke({"question": "원더카드가 뭐야", "answer": "", "score": 0.0, "retriever_docs": []})
print("\n[최종 답변]:", response["answer"])

앙상블리트리버에서 invoke 로 가져온 결과
 [Document(id='16eefc32-3bee-45c9-abdc-640091780395', metadata={'page': 4, 'source': 'b11463bc-c702-40b7-8730-a7a92051a060_card.pdf'}, page_content='# 2. 원더카드 서비스\n# 3) 서비스 영역별 가맹점 기준 및 유의사항'), Document(id='d279d281-3854-4aed-b4da-adce319426f3', metadata={'table': '| 서비스 영역 | 가맹점 기준 | 유의사항 |\n| --- | --- | --- |\n| 쿠팡 | 쿠팡 | ※상품권, 여행, 항공권, 티켓, 도서, 공연 매출은 서비스에서 제외됩니다. ※공식 홈페이지(온라인/모바일 앱)를 통한 결제 건에 한하여 서비스가 제공되며, 상품 선택 시 연결된 다른 사이트 및 앱 에서 결제한 경우와 오프라인 결제 건은 서비스에서 제외됩니다. ※롯데ON, 더현대닷컴은 공식 홈페이지 (온라인/모바일 앱) 결제 건에 한하여 서비스가 제공되며, 몰 이동을 통해 다른 계열사 및 온라인 쇼핑으로 이동하여 결제 하는 경우는 서비스에서 제외됩니다. |\n| 네이버쇼핑 | 네이버쇼핑 | ※상품권, 여행, 항공권, 티켓, 도서, 공연 매출은 서비스에서 제외됩니다. ※공식 홈페이지(온라인/모바일 앱)를 통한 결제 건에 한하여 서비스가 제공되며, 상품 선택 시 연결된 다른 사이트 및 앱 에서 결제한 경우와 오프라인 결제 건은 서비스에서 제외됩니다. ※롯데ON, 더현대닷컴은 공식 홈페이지 (온라인/모바일 앱) 결제 건에 한하여 서비스가 제공되며, 몰 이동을 통해 다른 계열사 및 온라인 쇼핑으로 이동하여 결제 하는 경우는 서비스에서 제외됩니다. |\n| 오픈마켓 | G마켓, 옥션, 11번가, 인터파크, 위메프, 티몬 | ※상품권, 여행, 항공권, 티켓, 도서, 공연 매출은 서비스에서 제외됩니다. ※공식 홈페이지(온라인/모바일 앱)를 통한 결제 건